# Sistema de Recomendación con Filtrado Colaborativo
## Filtrado Colaborativo

## Librería Surprise de Python
Es una librería de código abierto fácil de usar para construir y analizar Sistemas de Recomendación. Esta proporciona la mayoría de los algoritmos fundamentales para crear Sistemas de Recomendación basados en Filtrado Colaborativo. 

Algunas de las características proporcionadas por Surprise son:
- Da al usuario un perfecto control sobre sus experimentos 
- Documentación extensa y clara ofreciendo detalles precisos de cada algoritmo
- Una variedad de algoritmos de predicción listos para ser usados, como algoritmos baseline, métodos de vecindad (K-Neighbours), métodos basados en factorización de matrices (SVD, PMF, SVD++, NMF) y muchos más
- Diferentes medidas de similitud como Coseno, MSD (Mininum Square Difference) y Pearson entre otras
- Herramientas para evaluar, analizar y comparar el rendimiento de los algoritmos. Los procedimientos de validación cruzada pueden ser ejecutados muy fácilmente utilizando potentes iteradores de CV (inspirados en los que se pueden encontrar en scikit-learn), así como una búsqueda exhaustiva sobre un conjunto de parámetros
- Facilita la implementación de nuevas ideas de algoritmos de recomendación
- Alivia la tediosa tarea del manejo del conjunto de datos. Los usuarios pueden usar conjuntos de datos integrados (Movielens, Jester) y sus propios conjuntos de datos personalizados


## Importaciones para los Sistemas de Recomendación

In [15]:
import io
import pandas as pd 

from collections import defaultdict

from surprise import Dataset, Reader, SVD, BaselineOnly, NormalPredictor, KNNWithMeans, accuracy
from surprise.model_selection import cross_validate, train_test_split

A continuación se presentarán todas las direcciones donde estarán las bases de datos de las películas y los usuarios

In [17]:
data_path = 'dataset-movielens/u.data'
item_path = 'dataset-movielens/u.item'

## Análsis Exploratorio de Datos (EDA - Exploratory Data Analysis)
### Use a custom dataset

In [3]:
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(data_path, reader=reader)

cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9344  0.9440  0.9464  0.9468  0.9481  0.9439  0.0049  
MAE (testset)     0.7390  0.7472  0.7505  0.7501  0.7545  0.7483  0.0052  
Fit time          0.13    0.15    0.14    0.15    0.14    0.14    0.01    
Test time         0.04    0.08    0.04    0.08    0.04    0.06    0.02    


{'test_rmse': array([0.93443822, 0.9439795 , 0.9464343 , 0.94683459, 0.94805443]),
 'test_mae': array([0.73900625, 0.74720694, 0.75052416, 0.7500969 , 0.75445731]),
 'fit_time': (0.1293954849243164,
  0.15209650993347168,
  0.1439378261566162,
  0.1480998992919922,
  0.14281511306762695),
 'test_time': (0.04057788848876953,
  0.07998156547546387,
  0.03801560401916504,
  0.07990479469299316,
  0.03789973258972168)}

In [19]:
ratings_dict = {
  "itemID" : [ 1, 1, 1, 2, 2 ],
  "userID" : [ 9, 32, 2, 45, "user_foo" ],
  "rating" : [ 3, 2, 4, 3, 1 ],
}
dataframe = pd.DataFrame(ratings_dict)

reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df( dataframe[["userID", "itemID", "rating"]], reader=reader)

cross_validate(NormalPredictor(), data, cv=2)

{'test_rmse': array([2.46651072, 1.5258293 ]),
 'test_mae': array([2.16700811, 1.20933358]),
 'fit_time': (0.0, 0.0),
 'test_time': (0.0, 0.0009980201721191406)}

## Training Dataset 

In [4]:
trainset, testset = train_test_split(data, test_size=.15)

In [5]:
data.raw_ratings

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013'),
 ('62', '257', 2.0, '879372434'),
 ('286', '1014', 5.0, '879781125'),
 ('200', '222', 5.0, '876042340'),
 ('210', '40', 3.0, '891035994'),
 ('224', '29', 3.0, '888104457'),
 ('303', '785', 3.0, '879485318'),
 ('122', '387', 5.0, '879270459'),
 ('194', '274', 2.0, '879539794'),
 ('291', '1042', 4.0, '874834944'),
 ('234', '1184', 2.0, '892079237'),
 ('119', '392', 4.0, '886176814'),
 ('167', '486', 4.0, '892738452'),
 ('299', '144', 4.0, '877881320'),
 ('291', '118', 2.0, '874833878'),
 ('308', '1', 4.0, '887736532'),
 ('95', '546', 2.0, '879196566'),
 ('38', '95', 5.0, '892430094'),
 ('102', '768', 2.0, '883748450'),
 ('63', '277', 4.0, '875747401

In [18]:
def read_item_names():
  rid_to_name = { }
  name_to_rid = { }
  with io.open( item_path, 'r', encoding='ISO-8859-1' ) as f:
    for line in f:
      line = line.split('|')
      rid_to_name[line[0]] = line[1]
      name_to_rid[line[1]] = line[0]
  
  return rid_to_name, name_to_rid

rid_to_name, name_to_rid = read_item_names()


In [8]:
rid_to_name

{'1': 'Toy Story (1995)',
 '2': 'GoldenEye (1995)',
 '3': 'Four Rooms (1995)',
 '4': 'Get Shorty (1995)',
 '5': 'Copycat (1995)',
 '6': 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '7': 'Twelve Monkeys (1995)',
 '8': 'Babe (1995)',
 '9': 'Dead Man Walking (1995)',
 '10': 'Richard III (1995)',
 '11': 'Seven (Se7en) (1995)',
 '12': 'Usual Suspects, The (1995)',
 '13': 'Mighty Aphrodite (1995)',
 '14': 'Postino, Il (1994)',
 '15': "Mr. Holland's Opus (1995)",
 '16': 'French Twist (Gazon maudit) (1995)',
 '17': 'From Dusk Till Dawn (1996)',
 '18': 'White Balloon, The (1995)',
 '19': "Antonia's Line (1995)",
 '20': 'Angels and Insects (1995)',
 '21': 'Muppet Treasure Island (1996)',
 '22': 'Braveheart (1995)',
 '23': 'Taxi Driver (1976)',
 '24': 'Rumble in the Bronx (1995)',
 '25': 'Birdcage, The (1996)',
 '26': 'Brothers McMullen, The (1995)',
 '27': 'Bad Boys (1995)',
 '28': 'Apollo 13 (1995)',
 '29': 'Batman Forever (1995)',
 '30': 'Belle de jour (1967)',
 '31': 'Crimson Tide

In [9]:
name_to_rid

{'Toy Story (1995)': '1',
 'GoldenEye (1995)': '2',
 'Four Rooms (1995)': '3',
 'Get Shorty (1995)': '4',
 'Copycat (1995)': '5',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': '6',
 'Twelve Monkeys (1995)': '7',
 'Babe (1995)': '8',
 'Dead Man Walking (1995)': '9',
 'Richard III (1995)': '10',
 'Seven (Se7en) (1995)': '11',
 'Usual Suspects, The (1995)': '12',
 'Mighty Aphrodite (1995)': '13',
 'Postino, Il (1994)': '14',
 "Mr. Holland's Opus (1995)": '15',
 'French Twist (Gazon maudit) (1995)': '16',
 'From Dusk Till Dawn (1996)': '17',
 'White Balloon, The (1995)': '18',
 "Antonia's Line (1995)": '19',
 'Angels and Insects (1995)': '20',
 'Muppet Treasure Island (1996)': '21',
 'Braveheart (1995)': '22',
 'Taxi Driver (1976)': '23',
 'Rumble in the Bronx (1995)': '24',
 'Birdcage, The (1996)': '25',
 'Brothers McMullen, The (1995)': '26',
 'Bad Boys (1995)': '27',
 'Apollo 13 (1995)': '28',
 'Batman Forever (1995)': '29',
 'Belle de jour (1967)': '30',
 'Crimson Tide (1995

## Train Model
 

In [11]:
user_based_algorithm = KNNWithMeans( k = 50, sim_options= {
  'name': 'pearson_baseline',
  'user_based' : True
})

In [12]:
user_based_algorithm.fit( trainset=trainset )

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [13]:
user_id = str(196)
item_id = str(302)

print("Obteniendo la prediccion del usuario 196 para la pelicula", rid_to_name[item_id])
prediction = user_based_algorithm.predict( user_id, item_id, r_ui=4, verbose=True )
print(f"Prediccion: {prediction.est}")

Obteniendo la prediccion del usuario 196 para la pelicula L.A. Confidential (1997)
user: 196        item: 302        r_ui = 4.00   est = 4.16   {'actual_k': 50, 'was_impossible': False}
Prediccion: 4.158525008255024


In [16]:
# Predict ratings for all pairs (u, i) that are NOT in the training set.
number_of_predictions = 10
no_ratings_set = trainset.build_anti_testset()
predictions = user_based_algorithm.test(no_ratings_set)
# First map the predictions to each user.
top_n = defaultdict(list)
for uid, iid, true_r, est, _ in predictions:
  top_n[uid].append((iid, est))

# Then sort the predictions for each user and retrieve the k highest ones.
for uid, user_ratings in top_n.items():
  user_ratings.sort(key=lambda x: x[1], reverse=True)
  top_n[uid] = user_ratings[:number_of_predictions]

# Print recommended items for user 196
for (iid, rating) in top_n[user_id]:
  print("Película: {} - Valoración: {}".format(rid_to_name[iid], rating))

Película: Someone Else's America (1995) - Valoración: 5
Película: Hearts and Minds (1996) - Valoración: 5
Película: Angel Baby (1995) - Valoración: 5
Película: Incognito (1997) - Valoración: 5
Película: Aparajito (1956) - Valoración: 5
Película: Lamerica (1994) - Valoración: 5
Película: Nico Icon (1995) - Valoración: 5
Película: Zeus and Roxanne (1997) - Valoración: 5
Película: Aiqing wansui (1994) - Valoración: 5
Película: Great Day in Harlem, A (1994) - Valoración: 5
